In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from InterpolatingEFT.utils import loadConfig
from InterpolatingEFT.interpolator import rbfInterpolator

In [2]:
args = {}
args["configfile"] = "configs/default4D_fast.yaml"
os.chdir('c:\\Users\\tomru\\Documents\\PhD\\InterpolatingEFT')

In [3]:
config  = loadConfig(args["configfile"])
name = Path(args["configfile"]).stem

In [4]:
interp = rbfInterpolator()
interp.initialise(config["data"])

In [5]:
point = {'lchgXE3': [0.], 'lchwXE2': [1.], 'lctgreXE1': [2.], 'lchj3XE1': [3.]}
point_arr = np.array([[0., 1., 2., 3]])
print(interp.evaluate(pd.DataFrame(point)))
print(interp.evaluate_no_if(pd.DataFrame(point)))
print(interp.evaluate_no_pandas(point_arr))
print(interp.evaluate_no_if_no_pandas(point_arr))

12.084387747641003
12.084415658660987
12.084387747641003
12.084387747641003


In [6]:
%%timeit -n 2000 -r 7
interp.evaluate(pd.DataFrame(point))

1.24 ms ± 69.5 µs per loop (mean ± std. dev. of 7 runs, 2,000 loops each)


In [7]:
%%timeit -n 2000 -r 7
interp.evaluate_no_if(pd.DataFrame(point))

1.16 ms ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 2,000 loops each)


In [8]:
%%timeit -n 2000 -r 7
interp.evaluate_no_pandas(point_arr)

996 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 2,000 loops each)


In [9]:
%%timeit -n 2000 -r 7
interp.evaluate_no_if_no_pandas(point_arr)

1.02 ms ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 2,000 loops each)


In [22]:
# Testing scipy
from scipy.optimize import minimize

def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)

    return der
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

In [25]:
%%timeit -n 500 -r 7
res = minimize(rosen, x0, method='L-BFGS-B', bounds=((-1,1),(-1,1),(-1,1),(-1,1),(-1,1)))

4.27 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [26]:
%%timeit -n 500 -r 7
res = minimize(rosen, x0, method='L-BFGS-B', jac=rosen_der, bounds=((-1,1),(-1,1),(-1,1),(-1,1),(-1,1)))

1.23 ms ± 28.1 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
